In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
!pip install -q torch==1.4

ERROR: Could not find a version that satisfies the requirement torch==1.4 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torch==1.4


In [3]:
import requests
from PIL import Image
import torch
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from IPython.display import display_markdown
# from llama_index import SimpleDirectoryReader, GPTVectorStoreIndex

In [4]:
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

CUDA available: True
Device name: NVIDIA GeForce RTX 4080 SUPER


In [5]:
# Load model directly
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B")
# device = torch.device("cuda", torch.cuda.current_device())
# model = model.half().cuda()
# model.to(device)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
def generate_chat_template(model, tokenizer, system_prompt="", prompt="", 
                           temperature=0.7, top_p=0.9, max_new_tokens=1000, do_sample=True):
    # Define system and user prompts
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]

    # Create the text-generation pipeline
    pipeline_ = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

    # Manually format the prompt (since `apply_chat_template` is likely unsupported)
    formatted_prompt = system_prompt + "\n" + prompt

    # Set do_sample parameter logic
    if not do_sample:
        temperature = None
        top_p = None

    # Generate output
    outputs = pipeline_(
        formatted_prompt,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_p=top_p
    )

    # Extract and return the generated text
    generated_text = outputs[0]["generated_text"]
    return generated_text.split(formatted_prompt)[-1]  # Remove the prompt from the output


In [15]:
# Paraphrasing Prompt Basic Example
system_prompt = """
You are a basic paraphrasing tool.
"""

prompt = """
Paraphrase the following text:

Central Analytics Support & Enablement (CASE) is our enterprise support platform for the ~20k data scientists, data analysts, MLOps engineers, BI developers, and other support teams. CASE includes multiple features such as "CASEY" the chatbot, an intent-driven (i.e. non-Gen AI/non-LLM) chatbot microservice built on MS Copilot SDKs. Additionally, CASE includes the Collective Intelligence (CI) framework, conditional ticketing system, and MS Graph-enabled search. All of this enables product and support teams to efficiently address customer needs, manage their product knowledge documentation, and handle product-related issues.
"""

output = generate_chat_template(
    model=model,
    tokenizer=tokenizer,
    system_prompt=system_prompt,
    prompt=prompt,
    temperature=0.7,  # Allows for more variation
    top_p=0.9,        # Nucleus sampling
    do_sample=True    # Enables sampling
)
display_markdown(output, raw=True)


Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
